# TUT: Neural networks 101
> <span style="color:gray">
Original [Theano/Lasagne tutorial](https://github.com/DeepLearningDTU/nvidia_deep_learning_summercamp_2016/blob/master/lab1/lab1_FFN.ipynb) by 
Lars Maaløe ([larsmaaloee](https://github.com/larsmaaloee)),
Søren Kaae Sønderby ([skaae](https://github.com/skaae)), and 
Casper Sønderby ([casperkaae](https://github.com/casperkaae)). 
Converted to TensorFlow by 
Alexander R. Johansen ([alrojo](https://github.com/alrojo)), 
and updated by 
Toke Faurby ([faur](https://github.com/Faur)) and 
Mikkel Vilstrup ([mvilstrup](https://github.com/MVilstrup)).


In this notebook you will implement a simple neural network in TensorFlow utilizing the graph building and automatic differentiation engine of TensorFlow. 
In this notebook we'll go through a simple 2-D and 2-class classification problem using the **half-moon dataset**.
The half-moon dataset is a nice place to start as it offers simple visualization and the network can be trained quickly.


First we show how to implement the **logistic regression** in TensorFlow.
Logistic regression is linear and cannot solve the problem.
Then you will extend the model to a simple neural network (formally called a dense feedforward neural network), which is able to do much better, as it is able to capture non-linear relations.
In the bottom of the notebook there are some exercises to guide you through this.


Some details of TensorFlow can be a bit confusing, so don't get discouraged if you don't get it right away.
You'll pick them up when you worked with it for some time.


We assume that you are already familiar with backpropagation/gradient descent (if not please see 
[Andrej Karpathy](https://medium.com/@karpathy/yes-you-should-understand-backprop-e2f06eab496b) or 
[Michal Nielsen](http://neuralnetworksanddeeplearning.com/chap2.html)).

### TensorFlow Playerground

If you are new to Neural Networks, start by using the **[TensorFlow playground](http://playground.tensorflow.org/)** (very instructive visualization) to familiarize yourself with hidden layers, hidden units, activations, learning rate, etc.


In [ ]:
# Loading dependancies and supporting functions by running the code block below.
from __future__ import absolute_import, division, print_function 

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets
import tensorflow as tf

import os, sys
sys.path.append(os.path.join('.', '..'))
import utils

## The half-moon data set

Before begining to model it is always a good idea to explore the data.

In [ ]:
# Generate a dataset and plot it
np.random.seed(0)
num_samples = 300

X, y = sklearn.datasets.make_moons(num_samples, noise=0.20)

X_tr = X[:100].astype('float32')
X_val = X[100:200].astype('float32')
X_te = X[200:].astype('float32')

y_tr = y[:100].astype('int32')
y_val = y[100:200].astype('int32')
y_te = y[200:].astype('int32')

plt.scatter(X_tr[:,0], X_tr[:,1], s=40, c=y_tr, cmap=plt.cm.BuGn)

print(X.shape, y.shape)

num_features = X_tr.shape[-1]
num_output = 2

## Creating the graph

The building blocks of TensorFlow are variables and operations, with these we can form computational graphs.
Such as when computing the logistic regression which is implemented below:
$$y = nonlinearity(xW + b)$$

where $x$ is the input tensor, $y$ is the output tensor and $\{W, b\}$ are the weights (variable tensors).
The weights are initialized with an initializer of our choice.
* x has shape ```[batchsize, num_input]```. 
* ```W``` has shape ```[num_input, num_units]``` and 
* b has ```[num_units]```.
* y has then ```[batch_size, num_units]```.

In this case we will have 2 inputs, one for each dimension, and 2 output units, one for each class.

**NOTE**: to make building neural networks easier there are several high-level APIs that wraps TensorFlow functionality into high-level abstractions, such as layers.
In this first exercise we will use basic TensorFlow functions so that you can learn how to build it from scratch.
This will help you later if you want to build your own custom operations.

These ops will define edges along our computational graph. 
The below ops will compute a logistic regression, but can be modified to compute a neural network.

In [ ]:
# resets the graph, needed when initializing weights multiple times, like in this notebook
tf.reset_default_graph()

# Setting up placeholder, this is where your data enters the graph!
x_pl = tf.placeholder(tf.float32, [None, num_features], name='xPlaceholder')
# 'None' means that the size is determined at run time. This is commonly done for the 
# batch size.

## Define initializer for the weigths
# How the weights are initialized is VERY important for how well the network trains.
# We will look into this later, but for now we will just use a normal distribution.
weight_initializer = tf.truncated_normal_initializer(stddev=0.1)

## Set up variables
# These variables are weights in your network that can be update while running our graph.

### Create layer 1
with tf.variable_scope('layer1'): 
    W_1 = tf.get_variable('W', [num_features, num_output], # change num_output to 100 for mlp
                          initializer=weight_initializer)
    b_1 = tf.get_variable('b', [num_output], # change num_output to 100 for mlp
                          initializer=tf.constant_initializer(0.0))
    with tf.variable_scope('output'):
        
        l_1 = tf.matmul(x_pl, W_1) + b_1
        # The layer before the softmax should not have a nonlinearity
        # We will cover choosing nonlinearity later. For now use the linear rectifier
        # https://en.wikipedia.org/wiki/Rectifier_(neural_networks)

#         l_1 = tf.nn.relu(l_1)


### Create layer 2 (left as an exercise)
### Ignore this until you reach the exercises in the bottom.
# 1) add a non-linearity to layer 1 by uncommenting the last line
# 2) Change the dimensions of W_1 and b_1.
#     Notice, to make a hidden layer, the weights needs to have the following dimensionality
#     W[number_of_units_going_in, number_of_units_going_out]
#     b[number_of_units_going_out]
#     If we want to make a hidden layer with 100 units, we need to define the shape of the
#     first weight to W[2, 100], b[2] and the shape of the second weight to W[100, 2], b[2]
# 3) create layer 2. Look to how layer 1 was implemented for inspiration.
# 4) Change the definition of `y` to take layer 2 as input.


y = tf.nn.softmax(l_1)

### Examining the model

In [ ]:
# knowing how to print your tensors and ops is useful, here are some examples
print("---placeholders---")
print('x_pl')
print('Name:', x_pl.name)
print('Shape:', x_pl.get_shape())
print()
print("---weights---")
print('W_1')
print('Name:', W_1.name)
print('Shape:', W_1.get_shape())
print('b_1')
print('Name:', b_1.name)
print('Shape:', b_1.get_shape())
print()
print("---Tensors---")
print('l_1')
print('Name:', l_1.name)
print('Shape:', l_1.get_shape())

It is also possible to automatically print all the ops and variables by iterating through the graph.

In [ ]:
# using the graph to print ops
print("< operations >")
for op in tf.get_default_graph().get_operations():
    print(op.name)
print()

# variables are accessed through tensorflow
print("< variables >")
for var in tf.global_variables():
    print(var.name)


Notice that our weights and operations defined in the `layer1` space are saved in the `layer1` directory of the graph.

### Create the training ops
After we have built the network we have our tensors in our default [graph](https://www.tensorflow.org/api_docs/python/tf/Graph), which we can use to build the cost function and ops used for training.
Further, using our default graph we can print the operations and variables of our default graph.

In [ ]:
# y_ is a placeholder variable taking on the value of the target batch.
# This is used for computing the loss.
y_ = tf.placeholder(tf.float32, [None, num_output], name='yPlaceholder')

with tf.variable_scope('loss'):
    # computing cross entropy per sample
    cross_entropy = -tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1])

    # Average over samples
    # Averaging makes the loss invariant to batch size, which is very nice.
    cross_entropy = tf.reduce_mean(cross_entropy)

#### Optimzers
To train our neural network we need to update the parameters in direction of the negative gradient w.r.t the cost function we defined earlier.
This is called gradient descent.
We can use `tf.train.Optimizer` to get the gradients (using `compute_gradients`) for all parameters in the network w.r.t ``cross_entropy``.

*Intuition behind gradient descent*: Imagine that `cross_entropy` is a function and we want to go downhill.
We go downhill by changing the value of the paramters in direction of the negative gradient. 
Finally we can use the built-in `minimize` to calculate the stochastic gradient descent (SGD) update rule for each paramter in the network.


In [ ]:
with tf.variable_scope('trainOP'):
    # Defining our optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)

    # Computing our gradients
    grads_and_vars = optimizer.compute_gradients(cross_entropy)

    # Applying the gradients
    train_op = optimizer.apply_gradients(grads_and_vars)


    ## The three steps above can be compressed into one: 
    # train_op = optimizer.minimize(crossentropy)

Next, we define an op that computes the accuracy.
This is used to monitor the performance during training, but doesn't have any influence on training.

In [ ]:
with tf.variable_scope('performance'):
    # making a one-hot encoded vector of correct (1) and incorrect (0) predictions
    correct_prediction = tf.equal(tf.argmax(y, axis=1), tf.argmax(y_, axis=1))

    # averaging the one-hot encoded vector
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

The next step is to utilize our `train_op` function repeatedly in order to optimize our weights `W_1` and `b_1` to make the best possible linear seperation of the half moon dataset.

It is worth or read a short introduction on TensorFlow [sessions](https://www.tensorflow.org/versions/r0.10/api_docs/python/client.html#Session) before continuing to the next codeblock. Sessions are used to run TensorFlow graphs, they uses `fetches` to decide which parts of the graph to compute and `feed_dicts` to load data into the graph.

## Inspecting the graph

Before begining the training we will have a look at the graph.
Notice how the graph visualization is shaped by `tf.variable_scope`.


*Note*: The Jupyter inline graph visualizer is not as good as when TensorBoard is run in a browser, especially not for large models.

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    tmp_def = utils.rename_nodes(sess.graph_def, lambda s:"/".join(s.split('_',1)))
    utils.show_graph(tmp_def)

## Training
The next step is to utilize our `train_op` function repeatedly in order to optimize our weights (`W_1` and `b_1`) to make the best possible linear seperation of the half moon dataset.

It is worth or read a short introduction on TensorFlow [sessions](https://www.tensorflow.org/versions/r0.10/api_docs/python/client.html#Session) before continuing to the next codeblock. Sessions are used to run TensorFlow graphs, they uses `fetches` to decide which parts of the graph to compute and `feed_dicts` to load data into the graph.

In [ ]:
# defining a function to make predictions using our classifier
def pred(X_in, sess):
    # first we must define what data to give it
    feed_dict = {x_pl: X_in}
    # secondly our fetches
    fetches = [y]
    # utilizing the given session (ref. sess) to compute results
    res = sess.run(fetches, feed_dict)
    # res is a list with each indices representing the corresponding element in fetches
    return res[0]

In [ ]:
num_epochs = 1000

# restricting memory usage, TensorFlow is greedy and will use all memory otherwise
gpu_opts = tf.GPUOptions(per_process_gpu_memory_fraction=0.2)

train_cost, val_cost, train_acc, val_acc = [],[],[],[]
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_opts)) as sess:
    try:
        print('Begin training')
        # initializing all variables 
        init = tf.global_variables_initializer()
        sess.run(init)

        ## Plotting
        fig = plt.figure(figsize=(16,12))
        fig.add_subplot(321)
        plt.title("Untrained Classifier, training")
        utils.plot_decision_boundary(lambda x: pred(x, sess), X_tr, y_tr)
        fig.add_subplot(323)
        plt.title("Untrained Classifier, validation")
        utils.plot_decision_boundary(lambda x: pred(x, sess), X_val, y_val)

        for e in range(num_epochs):
            ### TRAINING ###
            # what to feed to our train_op
            # notice we onehot encode our predictions to change shape from (batch,) -> (batch, num_output)
            feed_dict_train = {x_pl: X_tr, y_: utils.onehot(y_tr, num_output)}

            # deciding which parts to fetch, train_op makes the classifier "train"
            fetches_train = [train_op, cross_entropy, accuracy]

            # running the train_op
            res = sess.run(fetches=fetches_train, feed_dict=feed_dict_train)
            # storing cross entropy (second fetch argument, so index=1)
            train_cost += [res[1]]
            train_acc += [res[2]]

            ### VALIDATING ###
            # what to feed our accuracy op
            feed_dict_valid = {x_pl: X_val, y_: utils.onehot(y_val, num_output)}

            # deciding which parts to fetch
            fetches_valid = [cross_entropy, accuracy]

            # running the validation
            res = sess.run(fetches=fetches_valid, feed_dict=feed_dict_valid)
            val_cost += [res[0]]
            val_acc += [res[1]]


            if e % 100 == 0:
                print("Epoch %i, Train Cost: %0.3f\tVal Cost: %0.3f\t Val acc: %0.3f" \
                      %(e, train_cost[-1],val_cost[-1],val_acc[-1]))

        ### TESTING ###
        # what to feed our accuracy op
        feed_dict_test = {x_pl: X_te, y_: utils.onehot(y_te, num_output)}

        # deciding which parts to fetch
        fetches_test = [cross_entropy, accuracy]

        # running the validation
        res = sess.run(fetches=fetches_test, feed_dict=feed_dict_test)
        test_cost = res[0]
        test_acc = res[1]
        print("\nTest Cost: %0.3f\tTest Accuracy: %0.3f"%(test_cost, test_acc))

        ## Plotting
        fig.add_subplot(322)
        plt.title("Trained Classifier, training")
        utils.plot_decision_boundary(lambda x: pred(x, sess), X_tr, y_tr)
        fig.add_subplot(324)
        plt.title("Trained Classifier, validation")
        utils.plot_decision_boundary(lambda x: pred(x, sess), X_val, y_val)

        epoch = np.arange(len(train_cost))
        fig.add_subplot(325)
        plt.title('Loss')
        plt.plot(epoch, train_cost,'r', label='Train Loss')
        plt.plot(epoch, val_cost,'b', label='Val Loss')
        plt.legend()
        plt.xlabel('Updates'), plt.ylabel('Loss')

        fig.add_subplot(326)
        plt.title('Accuracy')
        plt.plot(epoch, train_acc,'r', label='Train Accuracy')
        plt.plot(epoch, val_acc,'b', label='Val Accuracy')
        plt.legend(loc=4)
        plt.xlabel('Updates'), plt.ylabel('Accuracy')

        plt.tight_layout()
        plt.show()
    except KeyboardInterrupt:
        print('KeyboardInterrupt')

print('Done')

You have now seen how to use TensorFlow to setup and train a logistic regression classifier.
Now you will have to extend the code into a neural network.


# <span style="color:red"> Exercise 1: From logistic regression to 'deep learning'</span>

#### <span style="color:red"> EXE 1.1) Add a hidden layer. 
A linear logistic classifier is only able to create a linear decision boundary. 
Make the model into a nonlinear neural network by inserting a dense hidden layer between the input and output layers of the model.

The fully connected architecture we are trying to define can be visualized in the following manner: 

<img src="images/ffn.png" style="width: 500px;"/> 

Each hidden unit $h_i$ in each layer is connected to all the units in the former layer.
The weights of a layer can therefore be defined as a matrix $M$ with one row for every hidden unit in the former layer and one column for every unit in the current layer.
We also want to add a bias vector $b_i$ to each hidden unit $h_i$ (i.e. same number of elements as there are columns in the matrix).
Mathematically this equals the function: 

$$ Y = X * H + b $$

So in forward propagation we multiply each layer-matrix with its input and add a bias vector.
We then apply an activation function to the result and pass the output of this activation function as the input to the next layer.

1. **Modify the code in 'Creating the graph' by following the instructions in the comments.**
___
<span style="color:blue"> Answer: </span>


#### <span style="color:red"> EXE 1.2) Experiment with the number of hidden layers and number of units
 1. **What happens to the decision boundary?**
___
<span style="color:blue"> Answer: </span>


#### <span style="color:red"> EXE 1.3) Overfitting
When increasing the number of hidden layers / units the neural network will fit the training data better by creating a highly nonlinear decision boundary. If the model is to complex it will often generalize poorly to new data (validation and test set). 
  1. **How high can you get the training accuracy?**
  1. **Can you obseve this from the training and validation errors? **

___
<span style="color:blue"> Answer: </span>


#### <span style="color:red"> EXE 1.4) Changing optimizer
We used the vanilla stocastic gradient descent algorithm for parameter updates. This is usually slow to converge and more sophisticated pseudo-second-order methods usually works better. 
To use the other optimizers checkout which optimizers TensorFlow [supports](https://www.tensorflow.org/api_guides/python/train). 
In practice the ADAM algorithm (Kingma and Welling 2014 [Adam: A Method for Stochastic Optimization](https://arxiv.org/abs/1412.6980v9)) or RMSProp are the most commonly used.

**Setting the learning rate**: The learning rate is a hyperparameter, which can be tricky to set. The following values often work, and are good initial guesses, but you should always try a couple of different learning rates, to see which is best.


1. SGD + Momentum: learning rate 1.0 - 0.1 
2. ADAM: learning rate: 1e-4 - 1e-5
3. RMSPROP: somewhere between SGD and ADAM   

Heres a small animation comparing different optimization algorithms under hard circumstances.

Long Valley | Saddle Point
- | - 
<img src="http://i.imgur.com/2dKCQHh.gif?1" style="width: 400px;"/> | <img src="http://i.imgur.com/NKsFHJb.gif?1" style="width: 400px;"/>

Animations are from at http://imgur.com/a/Hqolp (which includes another example).

  1. **Try changing the optimizer to [ADAM](https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer), and compare with SGD. **
  1. **What are the differences in training time / performance?**
___
<span style="color:blue"> Answer: </span>
